# Are “custom loss functions” simple, or super hard?

Huh. I got stuck trying to use a “custom loss function in my `Find_Conspicuous_Disk` model. I had been lead to expect it to be pretty simple (as discussed here:[in the Keras doc](https://keras.io/api/losses/#creating-custom-losses) and [this post](https://towardsdatascience.com/how-to-create-a-custom-loss-function-keras-3a89156ec69b)). But I kept running into problems (as described in this [SO question](https://stackoverflow.com/questions/62393032/custom-loss-function-with-weights-in-keras)). 

(Worth noting that I actually was trying first to add a new **metric** so I could see the output of my new loss function while still learning with the original loss function MSE.)

So I decided the Keras doc was out of date, showing the old simple way not the complex new way. So I made a small test case in this notebook. **And it worked**. I figured something must be wrong, it can't be that easy. Probably it is just ignoring my loss function. So I modified it (multiplying `squared_difference` (below) by 1000) and indeed it still worked, but the reported loss values were just 1000 times bigger.

**Tracking loss history over 100 epochs:**
*   For size 10, the loss goes from **0.6554** down to **0.1070**
*   For size 1000, the loss goes from **0.5931** down to **0.0785**



In [1]:
%tensorflow_version 2.x
import tensorflow as tf

# Check for Colab Pro resources
def check_colab_resources():
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM'.format(ram_gb))
    if ram_gb < 20:
        print('Not using a high-RAM runtime.')
    else:
        print('Using a high-RAM runtime.')

check_colab_resources()

Not connected to a GPU
Your runtime has 13.6 gigabytes of available RAM
Not using a high-RAM runtime.


In [9]:
# Test custom loss function

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import random

def test(training_set_size=10):

    # Original version of my_loss_fn()
    def my_loss_fn(y_true, y_pred):
        squared_difference = tf.square(y_true - y_pred)
        return tf.reduce_mean(squared_difference, axis=-1)  # Note `axis=-1`

    # # A different version of my_loss_fn() -- which makes the loss 1000 times
    # # bigger -- to verify that this function is actually being used.
    # def my_loss_fn(y_true, y_pred):
    #     squared_difference = tf.square(y_true - y_pred) * 1000
    #     return tf.reduce_mean(squared_difference, axis=-1)  # Note `axis=-1`

    random_seed = 24304832
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    # Random training data of size train_set_count:
    #     training example, each as a pairs of float features on [0, 1]
    #     labels, each as a float on [0, 1]
    # train_set_count = 10
    # train_set_count = 1000
    X_train = np.random.random_sample((training_set_size, 2))
    y_train = np.random.random_sample((training_set_size, 1))

    print(X_train.shape)
    print(y_train.shape)

    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(2,)))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=my_loss_fn)
    # model.fit(X_train, y_train, epochs=100)
    model.fit(X_train, y_train, epochs=100, batch_size=32)

test(10)
# test(1000)

(10, 2)
(10, 1)
Epoch 1/100
1/1 [==============================] - 0s 337ms/step - loss: 0.6554
Epoch 2/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6412
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6273
Epoch 4/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6135
Epoch 5/100
1/1 [==============================] - 0s 7ms/step - loss: 0.5999
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 0.5867
Epoch 7/100
1/1 [==============================] - 0s 8ms/step - loss: 0.5737
Epoch 8/100
1/1 [==============================] - 0s 8ms/step - loss: 0.5609
Epoch 9/100
1/1 [==============================] - 0s 10ms/step - loss: 0.5484
Epoch 10/100
1/1 [==============================] - 0s 8ms/step - loss: 0.5361
Epoch 11/100
1/1 [==============================] - 0s 8ms/step - loss: 0.5240
Epoch 12/100
1/1 [==============================] - 0s 5ms/step - loss: 0.5121
Epoch 13/100
1/1 [=======================